In [31]:
# Import helper functions
%run helper.py

In [32]:
from sklearn.model_selection import train_test_split
driving_log = read_csv()
driving_log = clean_data(driving_log)
train_samples, validation_samples = train_test_split(driving_log, test_size=0.2)

Number of samples in input data: 2416
Number of samples going straight: 920
Number of random samples going straight that are kept ( 10.0 % ): 92
Number of samples without samples going straight: 1496
Without samples with large steering angles ( larger than +- 0.7 ): 1496
Number of samples with rescued samples:  1588


In [41]:
from keras.preprocessing.image import img_to_array
from skimage import transform, color


image, steering_angle = load_image(train_samples.iloc[21])
print(steering_angle)

image= img_to_array(image)
images = []
steering_angles = []
images.append(image)
steering_angles.append(steering_angle)
X_train = np.array(images)
y_train = np.array(steering_angles)
print(X_train.shape)
print(X_train[0,:,12,2])
print(y_train.shape)
print(y_train)

-0.12228599999999999
(1, 160, 320, 3)
[ 215.  248.  222.  221.  230.  217.  251.  232.  234.  234.  235.  236.
  238.  239.  240.  240.  244.  244.  246.  245.  247.  248.  250.  248.
  250.  251.  252.  253.  255.  255.  255.  255.  253.  255.  245.  239.
  255.  231.  242.  105.   93.   90.   58.   63.   57.   47.   84.   46.
   49.   75.  129.  128.   75.   51.   54.   61.  161.  120.  111.   95.
  120.  132.  190.  207.  191.  202.  193.  212.  186.  200.  201.  197.
  202.  183.   51.   53.    0.    4.   10.  143.  192.  126.   82.   90.
  106.  111.  114.  117.  111.  120.  117.  117.  108.  125.  163.  162.
  159.  147.  164.  144.  129.  122.  176.  115.   93.   77.  104.  112.
  151.   90.   94.  100.  132.  117.  142.  186.   92.  121.  145.  188.
  131.  141.  107.  107.  121.  137.  226.  185.  127.   91.  138.  113.
  114.  124.  155.  234.  151.  140.  155.  107.  160.  132.  119.  143.
  125.  127.  140.  128.  134.  228.  137.  122.  103.  112.  136.  116.
  143.  171. 

In [ ]:
# demonstrator of how generator works
import matplotlib.pyplot as plt
driving_log = read_csv()

for n in range(10):
    index = np.random.randint(0, 200)
    image, steering_angle = load_and_augment_image(driving_log.loc[index])
   
    plt.figure()

    plt.xticks([]), plt.yticks([])
    plt.imshow(image, cmap="gray")
    plt.title('Steer: ' + str(steering_angle))

    plt.show()

In [ ]:
driving_log = read_csv()
image, steering_angle = load_image(driving_log.loc[85], rand = 0.1)


plt.figure(figsize=(17,9))

for n in range(32):
    index = np.random.randint(0, 200)
    image, steering_angle = load_image(driving_log.loc[index], np.random.random())
    plt.subplot(4,8,n+1)
    plt.xticks([]), plt.yticks([])
    plt.imshow(image, cmap="gray")
    plt.title('Steer: ' + str(steering_angle))
    
plt.show()

#print(driving_log.loc[27])


# All below is testing stuff

In [ ]:
print(np.random.randint(3, size=5))

In [ ]:
driving_log = read_csv_pd()
print_histogram(driving_log['Steering Angle'].values)
driving_log = clean_data(driving_log, upper_angle=0.5, zero_frac=0.1)
print_histogram(driving_log['Steering Angle'].values)

In [ ]:
csv_data = read_csv()
train_samples, validation_samples = train_test_split(csv_data, test_size=0.2)

y_train = np.array(csv_data[3])
#X_train, y_train = import_pictures(csv_data)
#X_train, y_train = augment_pictures(X_train, y_train)

In [ ]:
# Keras data augmentation gen
datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('../SDCND_output/IMG/center_2017_07_18_12_37_19_118.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(X_train, batch_size=32,
                          save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 1:
        break  # otherwise the generator would loop indefinitely

In [ ]:
def import_pictures(csv_data, steering_correction=0.25):
    images = []
    steering_angle = []
    for line in csv_data:
        # Center image
        image_path = '../SDCND_output/IMG/' + line[0].split('\\')[-1]
        images.append(img_to_array(load_img(image_path)))
        steering_angle.append(float(line[3]))
        # Left image
        #image_path = '../SDCND_output/IMG/' + line[1].split('\\')[-1]
        #images.append(load_img(image_path))
        #steering_angle.append(float(line[3]) + steering_correction)
        # Right image
        #image_path = '../SDCND_output/IMG/' + line[2].split('\\')[-1]
        #images.append(load_img(image_path))
        #steering_angle.append(float(line[3]) - steering_correction)

    X_train = np.array(images)
    y_train = np.array(steering_angle)
    print('After import shape X_train: ', X_train.shape, ',  Shape y_train: ', y_train.shape)

    return X_train, y_train